# Data preparation

This notebook reads the `TXT` files of the speeches and builds a single dataframe with every tokenized and normalized content we'll use.

Please note that none of the code chunks of this notebook were actually ran from here. As the processing part took a great amount of time to be completed, we transformed this notebook in a `Python` script and submitted it through the slurm work manager of the Computer Science department. The script is the file `2_data_preparation.py` in this same directory, and the file `run_data_preparation.sbatch` loads it to the slurm environment.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import re
import spacy
from nltk.tokenize import sent_tokenize
#from lucem_illud_2020 import word_tokenize

import sys
sys.path.insert(0, '../../../scripts')
import data_cleaning as clean

Defining the directory of the speeches:

In [ ]:
speeches_dir = '../../../data/presidentialSpeechPeru/txt'

We load the corpus from this path using a helper function we created:

In [ ]:
speeches_raw = clean.loadcorpus(speeches_dir)

After this, we load the result in a data frame and start adding some metadata columns:

In [ ]:
speech = pd.DataFrame()
filenames = []
raw = []
for filename, raw_speech in speeches_raw.items():
    print(filename)
    filenames.append(filename)
    raw.append(raw_speech)
speech['filename'] = filenames
speech['raw text'] = raw

pattern = re.compile('[0-9]{4}')
speech['year'] = speech['filename'].apply(lambda x: pattern.search(x).group(0))
speech = speech.sort_values(by='year').reset_index(drop=True)

Now we clean these raw texts using another ad-hoc function:

In [ ]:
speech['cleaned text'] = speech['raw text'].apply(lambda x: clean.clean_raw_text(x))

Adding the administration and president of each speech:

In [ ]:
speech.loc[(speech['year'].astype('int32') >= 1956) & \
           (speech['year'].astype('int32') <= 1961), 'administration'] = 'Prado'

speech.loc[(speech['year'].astype('int32') >= 1962) & \
           (speech['year'].astype('int32') <= 1962), 'administration'] = 'Lindley'

speech.loc[(speech['year'].astype('int32') >= 1963) & \
           (speech['year'].astype('int32') <= 1968), 'administration'] = 'Belaunde(1)'

speech.loc[(speech['year'].astype('int32') >= 1969) & \
           (speech['year'].astype('int32') <= 1975), 'administration'] = 'Velasco'

speech.loc[(speech['year'].astype('int32') >= 1976) & \
           (speech['year'].astype('int32') <= 1979), 'administration'] = 'Morales Bermudez'

speech.loc[(speech['year'].astype('int32') >= 1980) & \
           (speech['year'].astype('int32') <= 1984), 'administration'] = 'Belaunde(2)'

speech.loc[(speech['year'].astype('int32') >= 1985) & \
           (speech['year'].astype('int32') <= 1989), 'administration'] = 'Garcia(1)'

speech.loc[(speech['year'].astype('int32') >= 1990) & \
           (speech['year'].astype('int32') <= 1994), 'administration'] = 'Fujimori(1)'

speech.loc[(speech['year'].astype('int32') >= 1995) & \
           (speech['year'].astype('int32') <= 2000), 'administration'] = 'Fujimori(2)'

speech.loc[(speech['year'].astype('int32') >= 2001) & \
           (speech['year'].astype('int32') <= 2005), 'administration'] = 'Toledo'

speech.loc[(speech['year'].astype('int32') >= 2006) & \
           (speech['year'].astype('int32') <= 2010), 'administration'] = 'Garcia(2)'

speech.loc[(speech['year'].astype('int32') >= 2011) & \
           (speech['year'].astype('int32') <= 2015), 'administration'] = 'Humala'

speech.loc[(speech['year'].astype('int32') >= 2016), 'administration'] = 'Kuzcynski/Vizcarra'

speech.loc[(speech['year'].astype('int32') >= 1956) & \
           (speech['year'].astype('int32') <= 1961), 'president'] = 'Prado'

speech.loc[(speech['year'].astype('int32') >= 1962) & \
           (speech['year'].astype('int32') <= 1962), 'president'] = 'Lindley'

speech.loc[(speech['year'].astype('int32') >= 1963) & \
           (speech['year'].astype('int32') <= 1968), 'president'] = 'Belaunde'

speech.loc[(speech['year'].astype('int32') >= 1969) & \
           (speech['year'].astype('int32') <= 1975), 'president'] = 'Velasco'

speech.loc[(speech['year'].astype('int32') >= 1976) & \
           (speech['year'].astype('int32') <= 1979), 'president'] = 'Morales Bermudez'

speech.loc[(speech['year'].astype('int32') >= 1980) & \
           (speech['year'].astype('int32') <= 1984), 'president'] = 'Belaunde'

speech.loc[(speech['year'].astype('int32') >= 1985) & \
           (speech['year'].astype('int32') <= 1989), 'president'] = 'Garcia'

speech.loc[(speech['year'].astype('int32') >= 1990) & \
           (speech['year'].astype('int32') <= 2000), 'president'] = 'Fujimori'

speech.loc[(speech['year'].astype('int32') >= 2001) & \
           (speech['year'].astype('int32') <= 2005), 'president'] = 'Toledo'

speech.loc[(speech['year'].astype('int32') >= 2006) & \
           (speech['year'].astype('int32') <= 2010), 'president'] = 'Garcia'

speech.loc[(speech['year'].astype('int32') >= 2011) & \
           (speech['year'].astype('int32') <= 2015), 'president'] = 'Humala'

speech.loc[(speech['year'].astype('int32') >= 2016) & \
           (speech['year'].astype('int32') <= 2017), 'president'] = 'Kuzcynski'

speech.loc[(speech['year'].astype('int32') >= 2018), 'president'] = 'Vizcarra'

speech['year-president'] = speech['year'] + '-' + speech['president']

In [ ]:
speech.head()

Tokenizing words:

In [ ]:
speech['tokenized_words'] = speech['cleaned text'].apply(lambda x: clean.word_tokenize(x))

Now we normalize:

In [ ]:
speech['normalized_words'] = speech['tokenized_words'].apply(lambda x: clean.normalize_tokens(x))

Now we check the most frequent words:

In [ ]:
countsDict = {}
for word in speech['normalized_words'].sum():
    word = word.lower()
    if word in countsDict:
        countsDict[word] += 1
    else:
        countsDict[word] = 1
word_counts = sorted(countsDict.items(), key = lambda x : x[1], reverse = True)
word_counts[:100]

Then, we tokenize sentences using the function from `nltk` for this:

In [ ]:
speech['tokenized_sentences'] = speech['cleaned text'].apply(sent_tokenize)

Now we tokenize each word in each sentence:

In [ ]:
speech['tokenized_words_in_sentences'] = speech['tokenized_sentences'].apply(lambda x: [clean.word_tokenize(s) for s in x])

Finally, we normalized each tokenized word within each sentence:

In [ ]:
speech['normalized_words_in_sentences'] = speech['tokenized_words_in_sentences'].apply(lambda x: [clean.normalize_tokens(s, stop_words) for s in x])

In [ ]:
speech.head()